In [ ]:
#TEST FAST API

In [7]:
import pandas as pd
import boto3
from io import StringIO
import psycopg2
import datetime

In [2]:
def connect_to_rds():
    try:
        connection = psycopg2.connect(
            user="postgres",
            password="postgres",
            host="db-tp-avanzada.c7o4g4e22k4y.us-east-1.rds.amazonaws.com",
            port="5432",
            database="postgres"
        )
        return connection
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
        return None

In [13]:
def get_recommendations(advertiser_id: int, model: str):
    # Conección a la base
    connection = connect_to_rds()
    if connection is None:
        return {"message": "Failed to connect to the database"}

    current_date = datetime.date.today()- datetime.timedelta(days=1)

    try:
        cursor = connection.cursor()

        # Defino la query en base al modelo
        if model == "TopProduct":

            query = f"SELECT product_id FROM top_product WHERE advertiser_id = '{advertiser_id}' AND date = '{current_date}' ORDER BY ranking LIMIT 20"
        
        elif model == "TopCTR":

            query = f"SELECT product_id FROM top_ctr WHERE advertiser_id = '{advertiser_id}' AND date = '{current_date}'' ORDER BY ranking LIMIT 20"
        
        else:
            return {"message": "Invalid model. Please choose either 'TopProduct' or 'TopCTR'"}

        cursor.execute(query)
        records = cursor.fetchall()

        if not records:
            return {"message": f"No recommendations found for advertiser_id {advertiser_id}"}

        products = [record[0] for record in records]
        return {"advertiser_id": advertiser_id, "recommended_products": products}
    
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
        return {"message": "Error occurred while fetching data from the database"}

    finally:
        if connection:
            cursor.close()
            connection.close()

In [14]:
get_recommendations('2WPF1NXECF3G6NUMWDO7', 'TopProduct')

{'advertiser_id': '2WPF1NXECF3G6NUMWDO7',
 'recommended_products': ['3oe469',
  '59j88y',
  '6gdowb',
  '8ldyn6',
  'mgktm4',
  'okk6di',
  'ugdfpk',
  '1kq04j',
  '79wnsa',
  '805ivg',
  '8s5t5t',
  '9cgdv8',
  'athjhi',
  'e8vxcd',
  'f9jep9',
  'gvu1el',
  'hc9qdl',
  'np11o4',
  'nrh896',
  'nt02vv']}

In [9]:
def get_history(advertiser_id: int):
    # Conección a la base
    connection = connect_to_rds()
    
    if connection is None:
        return {"message": "Failed to connect to the database"}

    try:
        cursor = connection.cursor()

        # Obtener la fecha actual
        current_date = datetime.date.today()

        # Definir la fecha límite para el historial (últimos 7 días)
        end_date = current_date
        start_date = end_date - datetime.timedelta(days=6)

        # Inicializar un diccionario para almacenar las recomendaciones para cada día
        history = {}

        for date in range((end_date - start_date).days + 1):
            # itero sobre las fechas
            date_iter = start_date + datetime.timedelta(days=date)

            # Definir la query para obtener las recomendaciones para este día
            query = f"SELECT product_id FROM top_product WHERE advertiser_id = '{advertiser_id}' AND date = '{date_iter}' ORDER BY ranking LIMIT 20"
            cursor.execute(query)
            records = cursor.fetchall()

            # Almacenar las recomendaciones para este día en el diccionario de historial
            if records:
                history[str(date_iter)] = [record[0] for record in records]
            else:
                history[str(date_iter)] = "no data for this date"

        return history
    
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
        return {"message": "Error occurred while fetching data from the database"}

    finally:
        if connection:
            cursor.close()
            connection.close()


In [10]:
get_history('2WPF1NXECF3G6NUMWDO7')

{'2024-04-30': 'no data for this date',
 '2024-05-01': ['r6ut7n',
  'zonvk2',
  '1gjnn4',
  '1za6l5',
  '2dklx7',
  '3lh107',
  '4t6u8i',
  '63pda8',
  '79wnsa',
  'athjhi',
  'cd6nzg',
  'cwaimx',
  'enci0h',
  'jdd2jx',
  'k2umzv',
  'k63v46',
  's831wm',
  'tecust',
  'urms05',
  'xb476y'],
 '2024-05-02': ['3lh107',
  'cd6nzg',
  'nrh896',
  '1gegxk',
  '1za6l5',
  '4g0a2g',
  '63pda8',
  '9xqcyx',
  'athjhi',
  'bs9awt',
  'eiohv0',
  'gkz6wc',
  'ha7fga',
  'ma6irv',
  'okk6di',
  'qr6buu',
  's831wm',
  'sc8wyj',
  'u8g108',
  'w3yjck'],
 '2024-05-03': ['7sizf8',
  'csdo4h',
  'f9jep9',
  'lohq9t',
  'nrh896',
  'qr6buu',
  '8yddnr',
  'awsnit',
  'cwaimx',
  'ee96fh',
  'eiohv0',
  'hc9qdl',
  'jcok02',
  'jgo4ep',
  'kkjuz8',
  'mgktm4',
  'np11o4',
  'nw5ji2',
  'nxm2xm',
  'p0dsnt'],
 '2024-05-04': ['8s5t5t',
  '9xqcyx',
  'e8vxcd',
  '0299ko',
  '1gjnn4',
  '4l9sze',
  '5u0fz3',
  '6m92v0',
  '7sizf8',
  '8tem7z',
  'cd6nzg',
  'eiohv0',
  'lq3nso',
  'ma6irv',
  'nljn8x',
 

In [27]:
def get_stats():
    # Conección a la base
    connection = connect_to_rds()
    if connection is None:
        return {"message": "Failed to connect to the database"}

    try:
        cursor = connection.cursor()

        # Fecha actual
        current_date = datetime.date.today()-datetime.timedelta(days=1)

        # Cantidad de advertisers activos
        cursor.execute("SELECT COUNT(DISTINCT advertiser_id) FROM top_product WHERE date = %s", (current_date,))
        num_advertisers = cursor.fetchone()[0]

        # Advertisers que más varían sus recomendaciones por día
        cursor.execute('''SELECT advertiser_id
                            FROM top_product
                            GROUP BY advertiser_id
                            ORDER BY COUNT(DISTINCT product_id) DESC
                            LIMIT 3''')
        advertisers_most_varied = cursor.fetchall()

        return {
            "num_advertisers": num_advertisers,
            "advertisers_most_varied": advertisers_most_varied
        }
    
    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
        return {"message": "Error occurred while fetching data from the database"}

    finally:
        if connection:
            cursor.close()
            connection.close()

In [28]:
get_stats()

{'num_advertisers': 20,
 'advertisers_most_varied': [('5E325T5HYL61QSABVR5V',),
  ('62FIK8F2YT8JSFDBLEC9',),
  ('6X20RDH567MX2X3TXYJ7',)]}